# Setup

In [ ]:
!pip install finlab_crypto > log.txt
!pip install talib-binary > log.txt

# Configuration

In [ ]:
# finlab_crypto >= 0.1.14 更新
# 不需要這段冗贅的程式碼囉！

# Define Strategy


In [ ]:
from finlab_crypto.strategy import Strategy
from finlab_crypto.indicators import trends

@Strategy(name='sma', n1=20, n2=40)
def trend_strategy(ohlcv):
  name = trend_strategy.name
  n1 = trend_strategy.n1
  n2 = trend_strategy.n2

  filtered1 = trends[name](ohlcv.close, n1)
  filtered2 = trends[name](ohlcv.close, n2)

  entries = (filtered1 > filtered2) & (filtered1.shift() < filtered2.shift())
  exit = (filtered1 < filtered2) & (filtered1.shift() > filtered2.shift())

  figures = {
      'overlaps': {
          'trend1': filtered1,
          'trend2': filtered2,
      }
  }
  return entries, exit, figures


# altcoin strategy
# --------------------
# 'XRPBTC', 'ADABTC', 'LINKBTC', 'ETHBTC', 'VETBTC'
# trend_strategy(ohlcv, variables={'name': 'sma', 'n1', 30, 'n2': 130}, freq='4h')
from finlab_crypto.online import TradingMethod, TradingPortfolio

tm1 = TradingMethod(
    symbols=['XRPUSDT', 'ADAUSDT', 'LINKUSDT', 'ETHBTC', 'VETBTC'],
    freq='4h',
    # lookback:backtest data length
    lookback=1500,
    strategy=trend_strategy,
    variables=dict(
      name='sma',
      n1=30,
      n2=130,
    ),
    filters={},
    # how to set weight_btc?Please see teacher additional materials.
    weight_btc=0.2,
    name='altcoin-trend-hullma'
)

# btc strategy
# --------------------
# 'BTCUSDT'
# trend_strategy(ohlcv, variables={'name': 'hullma', 'n1', 70, 'n2': 108}, freq='4h')

tm2 = TradingMethod(
    symbols=['BTCUSDT'],
    freq='4h',
    lookback=1500,
    strategy=trend_strategy,
    variables=dict(
        name='hullma',
        n1=70,
        n2=108,
    ),
    filters={},
    weight_btc=1,
    name='btc-trend-hullma'
)

# Build Trading Portfolio

In [ ]:
# Binance api key and secret
key = ''
secret = ''

tp = TradingPortfolio(key, secret)
tp.register(tm1)
tp.register(tm2)
tp.register_margin('USDT', 15000)

# Backtest Trading Portfolio

In [ ]:
import finlab_crypto
finlab_crypto.setup()

In [ ]:
# get all price data
full_ohlcvs = tp.get_full_ohlcvs()
results = tp.status(full_ohlcvs)

# Portfolio Backtest

In [ ]:
asset_returns = tp.portfolio_backtest(full_ohlcvs, '4h')

# Recent Signal


In [ ]:
ohlcvs = tp.get_ohlcvs()
signals = tp.get_latest_signals(ohlcvs)
signals

# Create new orders


In [ ]:
position, position_in_btc, new_orders = tp.calculate_position_size(signals)
position_in_btc

In [ ]:
new_orders

# Execute Orders


In [ ]:
# mode = 'LIMIT'
# mode = 'MARKET'
# mode = 'TEST'
order_results = tp.execute_orders(new_orders, mode='TEST')
order_results